<a href="https://colab.research.google.com/github/martaarozarena/KSchool-Master-Final-Project/blob/master/exog_varaibles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
import pandas as pd

In [145]:
sheets=["c1_schoolclosing","c2_workplaceclosing","c3_cancelpublicevents","c4_restrictionsongatherings","c5_closepublictransport","c6_stayathomerequirements","c7_domestictravel","c8_internationaltravel","h1_publicinfocampaign","h2_testingpolicy","h3_contacttracing"]
initialdate="01-01-2020"
enddate="30-06-2020"

In [146]:
var = pd.read_excel("https://github.com/OxCGRT/covid-policy-tracker/raw/master/data/timeseries/OxCGRT_timeseries_all.xlsx",sheet_name=sheets)

In [147]:
j=1
for i in var.keys():
  var[i]=var[i].drop("CountryCode",axis=1)
  var[i].index=var[i].CountryName
  var[i]=var[i].drop("CountryName",axis=1)
  var[i]=var[i][:-3]
  var[i]=var[i].T
  var[i].index=pd.to_datetime(var[i].index,format="%d%b%Y")
  var[i]=var[i].truncate(initialdate,enddate)
  j=j+1

In [148]:
b=1
for i in sheets:
  if b==1: 
    var[i].columns="{}_".format(i[3:])+var[i].columns
    exogenas=var[i]
  else:
    concat=var[i]
    concat.columns="{}_".format(i[3:])+var[i].columns
    exogenas=pd.concat([exogenas,concat],axis=1)
  b=b+1

In [ ]:
#download the mask wearing data

In [ ]:
url2 = "https://drive.google.com/file/d/1WsBkpNhB5ilLsNxmOZPwdXv66m5tPDHx/view?usp=sharing"
path2 = 'https://drive.google.com/uc?export=download&id='+url2.split('/')[-2]
var_percent_mask = pd.read_csv(path2,sep=";")

In [ ]:
var_percent_mask.DateTime=pd.to_datetime(var_percent_mask.DateTime)
var_percent_mask.index=var_percent_mask.DateTime
var_percent_mask.drop("DateTime",axis=1,inplace=True)

In [ ]:
#add the missing dates
var_percent_mask=var_percent_mask.resample("D").mean()

In [ ]:
#fill the missing dates with an interpolation
var_percent_mask.interpolate(method='linear',inplace=True)
var_percent_mask.fillna(method="bfill",inplace=True)


In [ ]:
#rename the variable to be added in the general dataframe and choose the dates
var_percent_mask.columns="mask_"+var_percent_mask.columns
var_percent_mask=var_percent_mask.truncate(initialdate,enddate)

In [ ]:
#added to the generales dataframe and the missing initial values add 0s cause people was not wearing masks at the beggining
exogenas=pd.concat([exogenas,var_percent_mask],axis=1)
exogenas.fillna(0,inplace=True)

In [ ]:
#creating the csv file
exogenas.to_csv('exogenas.csv')

In [ ]:
exogenas.shape

(182, 1947)